Notebook version 1.0, 31 Aug 2021. Written by Otto Salmenkivi / CSC - IT Center for Science Ltd. otto.salmenkivi@gmail.com

Licensed under the MIT license: http://www.opensource.org/licenses/mit-license.php

Tested on Kvasi, running QLM version 1.2.1: https://research.csc.fi/-/kvasi
***
# Quantum Algorithm for calculating the expected value from a probability distribution

In this notebook we employ the [Quantum Amplitude Estimation][1] (QAE) algorithm to estimate the expected value of a probability distribution. The QAE algorithm can theoretically provide a quadratic speed-up compared to similar classical calculations. The algorithm consist of the following parts:
- Distribution loading with $D$ gate to `nb_dist_qbits` number of qubits
- $F$ gate representing the objective function
- A Quantum Phase Estimation protocol that utilizes `nb_eval_qbits` number of qubits

We have a discrete variable $i \in \{0,1,2,3\}$, that we map to quantum states $|00\rangle$, $|01\rangle$, $|10\rangle$ and $|11\rangle$. The values and therefore the states have probabilities $\frac{1}{2}$, $\frac{1}{4}$, $\frac{1}{6}$ and $\frac{1}{12}$, respectively. We can easily calculate the expected value:
\begin{equation}
    \mathbb{E}[i] = \sum_i p_i\cdot i = \dfrac{1}{2}\cdot 0 + \dfrac{1}{4} \cdot 1 + \dfrac{1}{6} \cdot 2 + \dfrac{1}{12} \cdot 3 \approx 0,83\,.
\end{equation}
Our task is to approximate the value above with a quantum algorithm.

[1]: https://arxiv.org/abs/quant-ph/0005055

We begin by importing the libraries needed

In [ ]:
from qat.lang.AQASM import Program, QRoutine, AbstractGate, H, SWAP
from qat.lang.AQASM.qftarith import QFT
from qat.qpus import LinAlg
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Defining number of qubits used in the algorithm
nb_eval_qbits = 6
nb_dist_qbits= 2 # Cannot be changed because Distribution is hand-build for 2 qubits

# Initializing abstract Distribution, F and Q gates
D_gate = AbstractGate("D", [], arity = nb_dist_qbits)
F_gate = AbstractGate("F", [], arity = nb_dist_qbits + 1)
Q_gate = AbstractGate("Q", [int], arity = nb_dist_qbits + 1)

# Constants
c = 1/2    # c is a constant used in a linear approximation in the QAE algoritm, defined within (0,1]

## D gate: Loading the uncertainty distribution to 2 qubits with parametrized RY gate matrices  
The integers $i \in \{0,1,2,3\}$ are mapped to a register of two qubits so that $i = 2i_0 + i_1$. Indexing follows the QLM convention.

We create matrices corresponding to the $R_y$ gates and calculate the resulting matrix, we call it $D$ gate. The values for the rotation angles were chosen to correspond with probabilities mentioned earlier.

In [ ]:
# Matrix of the standard RY gate
def ry_matrix(theta):
    return np.array([[np.cos(theta/2),-np.sin(theta/2)],[np.sin(theta/2),np.cos(theta/2)]])

# The resulting matrix of two RY gates acting on q_0 and q_1 respectively
def D_matrix():
    theta0 = 2*np.arccos(np.sqrt(2/3))
    theta1 = np.pi/3
    matrix = np.kron(ry_matrix(theta1), ry_matrix(theta0))
    return matrix

# Attach matrix to gate
D_gate.set_matrix_generator(D_matrix)

## F gate 
Next task is to build the matrix for the $F$ gate that acts on the objective qubit controlled by the distribution qubits.  The key is to flip the objective qubit with parametrised $R_y$-gates. The goal is to change the probability of measuring the objective qubit in state $|1\rangle$, let's call it $a$.

Because we are approximating the expected value of the variable itself, we define our objective function as $f(i) = i$. For the F operator though, we need to normalise it to $[0,1]$, so we get $\tilde{f}(i) = \dfrac{i}{3}$. 

For more information on building the circuit for the objective function, see [Stamatopoulus et al. 2020][1]. Here are the highlights:

With the $F$ operator we want to create the following tranformation:
\begin{equation}
    |i_0i_1\rangle|0\rangle \rightarrow |i_0i_1\rangle\big(\cos[f(i)]|0\rangle + \sin[f(i)]|1\rangle\big).
\end{equation}
Register $|i_0i_1\rangle$ is used for the distribution and the objective qubit is prepared in the zero state. Instead of using $\tilde{f}(i)$, we need to set the function as
\begin{equation}
    f(i) = c\left(2\cdot\dfrac{i}{3}-1 \right) + \dfrac{\pi}{4}\,,
\end{equation}
where $c \in (0,1]$ is a constant used in a linear approximation. By using $i = 2i_0 + i_1$, we get
\begin{equation}
    \dfrac{4c}{3}i_0 + \dfrac{2c}{3}i_1 - c + \dfrac{\pi}{4} \,.
\end{equation}
The operator is constructed from $R_y$ gates. The constant term is achieved by operating with $R_y\left(-c + \frac{\pi}{4}\right)$ on the objective qubits. The linear terms are implemented with controlled gates. A $R_y\left(\frac{4c}{3}\right)$ gate, controlled by $i_0$, and a $R_y\left(\frac{2c}{3}\right)$ gate, controlled by $i_1$, are applied on the objective.

The reason for this procedure is that with the probability to measure the last qubit in state $|1\rangle$ is related to the expected value of the objective function. We have
\begin{align}
    a &= \mathbb{E}[f(i)] = \mathbb{E}\left[c\left(2\cdot\dfrac{i}{3}-1 \right) + \dfrac{\pi}{4}\right]\\
    \Leftrightarrow \quad \mathbb{E}[i] &= \dfrac{3}{2}\left(\dfrac{a-\frac{1}{2}}{c} + 1 \right)\,.
\end{align}


We could easily build the $F$ operator with individual gates, but we choose to use matrix definitions and the `AbstractGate` class instead. This is because we need to have the matrix representation when building the $A$ and $Q$ gates in the following sections


[1]: https://arxiv.org/pdf/1905.02666.pdf


In [ ]:
# A 8x8 matrix that correspond to identity on first two qubist and a Ry on the last
def only_ry(theta):
    return np.kron(np.identity(4),ry_matrix(2*theta)) # The angle is doubled due to different definitions on RY matrix

# For the second gate matrix we need a matrix representation of a controlled RY,
# where the qubits are not adjecent
def cry_0to2(theta):
    theta = 2*theta # Again, doubled due to different matrix definitions
    matrix = np.array([[1,0,0,0,0,0,0,0],
                       [0,1,0,0,0,0,0,0],
                       [0,0,1,0,0,0,0,0],
                       [0,0,0,1,0,0,0,0],
                       [0,0,0,0,np.cos(theta/2),-np.sin(theta/2),0,0],
                       [0,0,0,0,np.sin(theta/2),np.cos(theta/2),0,0],
                       [0,0,0,0,0,0,np.cos(theta/2),-np.sin(theta/2)],
                       [0,0,0,0,0,0,np.sin(theta/2),np.cos(theta/2)]])
    return matrix

# controlled RY when most significant is control
def cry_1to2(theta):
    theta = 2*theta # Again, doubled due to different matrix definitions
    cry_matrix= np.array([[1,0,0,0],
                          [0,1,0,0],
                          [0,0,np.cos(theta/2),-np.sin(theta/2)],
                          [0,0,np.sin(theta/2),np.cos(theta/2)]])
    matrix = np.kron(np.identity(2),cry_matrix)
    return matrix

# combining all three
def F_matrix():
    f_1 = 2*c/3
    f_0 = - c + np.pi/4
    matrix = np.matmul(cry_0to2(2*f_1),np.matmul(cry_1to2(f_1),only_ry(f_0)))
    return matrix

# attach to gate
F_gate.set_matrix_generator(F_matrix) 

## A gate
Before the $Q$ gates, we need calculate the matrix $A$, which is the effective operator of both $D$ and $F$ gate on the objective and the distribution qubits. That is
\begin{equation}
    A|q_{0}q_{1}q_{obj}\rangle = F \cdot (D \otimes I)|q_0q_1q_{obj}\rangle\,.
\end{equation}

In [ ]:
def make_A_matrix():
    return np.matmul(F_matrix(),np.kron(D_matrix(),np.identity(2)))

To get the statevector $|\psi\rangle$ needed in the contruction of $Q$, we can run the $A$ gate separately an measure only the distribution qubits.

In [ ]:
# Run a program with only A gate
sv_prog = Program()
sv_qbits = sv_prog.qalloc(nb_dist_qbits+1)
A_gate = AbstractGate("A",[],arity = nb_dist_qbits+1,matrix_generator = make_A_matrix)
sv_prog.apply(A_gate(),sv_qbits)
A_result = LinAlg().submit(sv_prog.to_circ().to_job())

# get statevector from results
sv = A_result.statevector
print(f'Whole statevector after A gate:\n{sv}')

# Extract state correponding with last qubit states 0 and 1
psi_0 = []
psi_1 = []
for i in range(2**(nb_dist_qbits)):
    psi_0.append(sv[2*i])
    psi_1.append(sv[2*i+1])

a = sum([abs(i)**2 for i in psi_1])
print('The probability to measure the last qubit in state 1 based in the statevector is ',a)

print('Full measurement results after A gate:')
plt.figure()
states = [str(sample.state) for sample in A_result]
probs = [sample.probability for sample in A_result]
plt.bar(states,probs)
plt.ylabel('Probability')
plt.xticks(rotation = 60)
plt.draw()

At this point, we can already check the results from our quantum circuit, because we have a value for $a$, which we can trace back to the expected value.

This can be considered cheating, because this method does not provide a computatinal speed up. The point of the QAE algorithm is to get an approximation in a way that the approximation error converges quadratically faster than just by repeating the measument for the $A$ gate. But anyway, since our circuits are so small, we can use this cheat as a comparison.

In [ ]:
# Calculating Expected value based on value a that we can recover in the previous cell
def ExpectedFx(a):
    return 3*((a-1/2)/c+1)/2

print('Expected value for V from repeated A gates:',ExpectedFx(a),'.\n'
     'It is quite close to the true value.')


## The QPE and the Q gates

Quantum Phase estimation is well used tool in the world of quantum algorithms. For it we need additional evaluation qubits, the $Q$ gate and an inverse Quantum Fourier Transform. In addition, we need to flip the register before the inverse QFT.

The $Q$ gate is defined as $Q = AS_0A^{\dagger}S_{\psi_0}$, where $S_0 = I-2|0\rangle_{n+1}\langle0|_{n+1}$ and $S_{\psi_0} = I-2|\psi_0\rangle_n|0\rangle\langle\psi_0|_n\langle0|$ are reflections in the statevector space to amplify the "good" states, defined by the last qubit in state $|1\rangle$. Different powers of $Q^{2^j}$ gates are applied to the distribution and objective registers, when $j$ is the index of the control qubit in the evaluation register (reversed in the QLM convention).

In [ ]:
# Creating a matrix representation of Q based on Woerner & Egger 2018 Appendix A
def Q_matrix(j):
    A_matrix = make_A_matrix()
    statevector_psi = psi_0
    
    # We renormalize psi_0
    n = len(statevector_psi)
    statevector_sum = sum([abs(i)**2 for i in statevector_psi])
    for i in range(n):
        statevector_psi[i]  = statevector_psi[i]/np.sqrt(statevector_sum)
    
    # Preparing the needed states |0> and <0| as numpy arrays
    ket_zero = np.array([[1],[0]],dtype=complex)
    bra_zero = np.conjugate(np.transpose(ket_zero))
    
    #Preparing states |0>_n and <0|_n
    ket_zero_n = np.zeros((2*n,1),dtype= complex)
    ket_zero_n[0][0] = 1
    bra_zero_n = np.conjugate(np.transpose(ket_zero_n))
    
    # States |psi> and <psi| as numpy array based on input statevector
    ket_psi = np.zeros((n,1), dtype=complex)
    for i,amp in enumerate(statevector_psi):
        ket_psi[i][0] = amp
    
    bra_psi = np.conjugate(np.transpose(ket_psi))
    

    # Calculating the parentheses on the Q operator equation
    I = np.identity(2*n)
    first_parenthesis = I-2*np.matmul(ket_zero_n,bra_zero_n)
    second_parenthesis = I-2*np.matmul(np.kron(ket_psi,ket_zero),np.kron(bra_psi,bra_zero))
    
    # The Hermitian conjugate of operator A
    A_matrix_dag = np.conjugate(np.transpose(A_matrix))
    
    # Calculating the power based on parameter j
    single_matrix = np.matmul(A_matrix,np.matmul(first_parenthesis,np.matmul(A_matrix_dag,second_parenthesis)))
    Q_matrix = single_matrix
    i = 0
    while i < 2**j-1:
        Q_matrix = np.matmul(single_matrix,Q_matrix)
        i += 1
    return Q_matrix

# Attach to gate
Q_gate.set_matrix_generator(Q_matrix)

In [ ]:
# Do SWAPS
def swap_register(m):
    rout = QRoutine()
    i = 1
    while i <= m/2:
        rout.apply(SWAP, i-1, m-i)
        i += 1
    return rout

In [ ]:
# function to apply Hadamard gates multiple qubits
def hadamard_all(nbqbits):
    rout = QRoutine()
    for i in range(nbqbits):
        rout.apply(H,i)
    return rout

## It's time to build the whole circuit.
We have defined all the pieces we need, so let's build the circuit.

In [ ]:
def create_the_routine(nb_eval_qbits,nb_dist_qbits):
    rout = QRoutine()
    # Distribution qubits
    dist_qbits = rout.new_wires(nb_dist_qbits)    
    # The objective qbit
    obj_qbit = rout.new_wires(1)
    # Evaluation qubits
    eval_qbits = rout.new_wires(nb_eval_qbits)
    # Hadamards to evaluation qubits
    rout.apply(hadamard_all(nb_eval_qbits),eval_qbits)
    # D gate to distribution qubits based on distribution parameters
    rout.apply(D_gate(), dist_qbits)
    # F gate to objective and distribution qubits
    rout.apply(F_gate(),dist_qbits, obj_qbit)
    # A range of Q qates
    for j in range(nb_eval_qbits):
        rout.apply(Q_gate(j).ctrl(), eval_qbits[nb_eval_qbits-1-j], dist_qbits, obj_qbit)
    # Flip the register
    if nb_eval_qbits > 1 : rout.apply(swap_register(nb_eval_qbits), eval_qbits)
    # Apply inverse QFT
    rout.apply(QFT(nb_eval_qbits).dag(),eval_qbits)
    return rout

In [ ]:
prog = Program()
qbits = prog.qalloc(nb_eval_qbits+nb_dist_qbits+1)
prog.apply(create_the_routine(nb_eval_qbits,nb_dist_qbits),qbits)
circuit = prog.to_circ()
%qatdisplay circuit

Let's print the results from simulating the circuit above with a LinAlg simulator.

In [ ]:
result = LinAlg().submit(circuit.to_job(nbshots = 0, qubits = range(3,3+nb_eval_qbits)))
for sample in result:
    print(f'State: {sample.state},  amplitude: {sample.amplitude},  probability: {sample.probability}')
    
# Visualizing the initial measurement results
import matplotlib.pyplot as plt
plt.figure()
states = [str(sample.state) for sample in result]
probs = [sample.probability for sample in result]
plt.bar(states,probs)
plt.ylabel('Probability')
plt.xlabel('States')
plt.xticks(rotation = 60)
plt.draw()

## Post-prosessing the measurement results

The measurent gives integers $y$ which need to be mapped to an estimator $\tilde{a} = \sin^2(y\pi/M) \in [0,1]$.

In [ ]:
# Creating an array of zeros for all state probabilities and replacing the non-zero values for measured probabilities 
all_probs = np.zeros(2**nb_eval_qbits, dtype=float)
for sample in result:
    state_decimal = sample.state.int
    all_probs[state_decimal] = sample.probability

# The mapping used between measured states and corresponding estimate for the probability p
a_tildes = [np.sin(i*np.pi/(2**nb_eval_qbits))**2 for i in range(2**(nb_eval_qbits-1)+1)]

# Aggregating the data from different states that correspond to same probability bins
probs =[]
probs.append(all_probs[0])
i = 1
while i < 2**nb_eval_qbits/2:
    #print(f'These states correspond to the same bin: {i} and {2**nb_eval_qbits-i}')
    probs.append(all_probs[i] + all_probs[2**nb_eval_qbits-i])
    i += 1
probs.append(all_probs[2**(nb_eval_qbits-1)])

# finding a with highest probability
lucky_a_tilde = a_tildes[np.argmax(probs)]
print(f'The algorithm gives an estimate a = {lucky_a_tilde}')

We can visualise the end results by plotting the discrete probability distribution of $\tilde{a}$. We can also display the same distribution as a function of $\mathbb{E}[i]$ and show the true value (red line).

In [ ]:
plt.figure()
plt.bar(a_tildes, probs, width = 0.025)
plt.ylabel('Probability')
plt.xlabel('Estimator for a')
plt.plot()

plt.figure()
plt.bar([ExpectedFx(a_tilde) for a_tilde in a_tildes], probs, width = 0.05)
plt.ylabel('Probability')
plt.xlabel(r'E[i]')
plt.axvline(x=0.8333, color = 'r', linestyle = '--', linewidth = 2 )
plt.xlim(0,4)
plt.plot()

print(f'The best approximation for expected value for i = {ExpectedFx(lucky_a_tilde)}')

## The end
And there you have it!  We started from a probability distribution for a discrete variable and calculated the expected value for it using the QAE algorithm. Next step could be to approximate some linear function based on the variable, and mayde make it more interesting by using multiple variables each with a different probability distributions.